In [1]:
import os
import glob
import pickle
import math

import numpy as np
from tqdm.auto import tqdm
from joblib import Parallel, delayed

In [2]:
file_pat = "/kaggle/input/ppg-dalia-dataset/PPG_FieldStudy/S*/S*.pkl"
filepaths = glob.glob(file_pat)

In [3]:
with open(filepaths[0], "rb") as f:
    data = pickle.load(f, encoding = "latin-1")


In [4]:
print(data.keys())
activities = data["activity"]
print(len(activities))

dict_keys(['rpeaks', 'signal', 'label', 'activity', 'questionnaire', 'subject'])
35832


In [5]:
for path in tqdm(filepaths):
# for path in filepaths:
    subject = path.split("/")[-2]

    with open(path, "rb") as f:
        data = pickle.load(f, encoding="latin-1")
        
    ecg = data["signal"]["chest"]["ECG"].astype(np.float32).flatten()
    ppg = data["signal"]["wrist"]["BVP"].astype(np.float32).flatten()
    acc = data["signal"]["wrist"]["ACC"].astype(np.float32)
    acc = np.transpose(acc, (1, 0))
    
    activities = data["activity"].astype(np.float32).flatten()
    label = data["label"].astype(np.float32).flatten()
    
    num_labels = label.shape[0]

    freq_s = 700
    ecg_chunks = []
    assert math.floor(ecg.shape[0] / 700 / 2 - 3) == num_labels

    for i in range(num_labels):
        start = i * freq_s * 2
        end = start + 8 * freq_s
        ecg_chunks.append(ecg[start:end])

    freq_s = 64
    ppg_chunks = []
    assert math.floor(ppg.shape[0] / 64 / 2 - 3) == num_labels

    for i in range(num_labels):
        start = i * freq_s * 2
        end = start + 8 * freq_s
        ppg_chunks.append(ppg[start:end])

    freq_acc = 32
    acc_chunks = []
    assert math.floor(acc.shape[1] / 32 / 2 - 3) == num_labels
    for i in range(num_labels):
        start = i*freq_acc * 2
        end =  start + 8 * freq_acc
        acc_chunks.append(acc[:,start:end])
    
    
    
    freq_act = 4
    activity_chunks = []
    assert math.floor(activities.shape[0] / 4 / 2 - 3) == num_labels
    for i in range(num_labels):
        start = i * freq_act * 2
        end = start + 8 * freq_act
        activity_chunks.append(activities[start:end])
    
    
    

    ecg_chunks = np.vstack(ecg_chunks)
    ppg_chunks = np.vstack(ppg_chunks)
    acc_chunks  = np.stack(acc_chunks, axis=0)
    activity_chunks = np.vstack(activity_chunks)
    
    np.savez_compressed(subject, ecg=ecg_chunks, ppg=ppg_chunks,acc=acc_chunks, label=label, activity = activity_chunks)
    loaded = np.load(subject+'.npz', mmap_mode='r')
    for k in loaded.files:
        print(k)
    

  0%|          | 0/15 [00:00<?, ?it/s]

ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity
ecg
ppg
acc
label
activity


In [6]:
acc_chunks.shape

(3954, 3, 256)